In [17]:
basedir = "../.."
data_file = "experiments/pilot_random1_player_round_slim.csv"
device = "cpu"
experiment_names = ["trail_rounds_2"]
# experiment_names = ["random_1"]
# dataset_folder = "data/transformer_clone/random_1_v2"
dataset_folder = "data/transformer_clone/trail_rounds_2_v2"

In [18]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
# from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
from itertools import permutations
from sklearn.model_selection import train_test_split
from aimanager.transformer.dataset import MyDataset


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
def create_decayed_round_feature(df, decay_start, decay_end):
    decay_range = decay_end - decay_start
    df.loc[df['round_number'] < decay_start, 'decay_feature'] = 1
    df.loc[df['round_number'] >= decay_end, 'decay_feature'] = 0
    df.loc[(df['round_number'] >= decay_start) & (df['round_number'] < decay_end), 'decay_feature'] = 1 - ((df['round_number'] - decay_start) / decay_range)
    return df


df = pd.read_csv(os.path.join(basedir, data_file))

df = df[df['experiment_name'].isin(experiment_names)]

# Get all unique player ids
unique_player_ids = df['player_id'].unique()

# Generate all permutations of player ids
player_permutations = list(permutations(unique_player_ids))

# Create an empty dataframe to store augmented data
augmented_df = pd.DataFrame()

# Iterate over all permutations
for i, perm in enumerate(player_permutations):
    
    # Create a copy of the original dataframe
    df_copy = df.copy()
    
    # Create a new column for permuted player ids
    df_copy['new_player_id'] = df_copy['player_id'].apply(lambda x: perm[unique_player_ids.tolist().index(x)])
    
    # Add a column indicating permutation group
    df_copy['perm_group'] = i

    # Concatenate the permuted dataframe to the augmented dataframe
    augmented_df = pd.concat([augmented_df, df_copy])

# Reset the index
augmented_df = augmented_df.reset_index(drop=True)

# Use the augmented dataframe from now on
df = augmented_df

# Create dummy variables for 'new_player_id'
df = pd.concat([df, pd.get_dummies(df['new_player_id'], prefix='player_id')], axis=1)

# Add cyclic encoding for round_number
df['round_number_sin'] = np.sin(2*np.pi*df['round_number']/6)
df['round_number_cos'] = np.cos(2*np.pi*df['round_number']/6)

# Add linearly decaying feature for the first three rounds
df = create_decayed_round_feature(df, 0, 3) 

# scale 'contribution' and 'punishment'
df[['contribution_scaled', 'punishment_scaled']] = df[['contribution', 'punishment']] / 20

# Now, let's create the events
df_contribution = df.copy()
df_contribution['event_type'] = 'contribution'
df_contribution['event_id'] = 0
df_contribution.rename(columns={'contribution': 'event_value_raw', 'contribution_scaled': 'event_value'}, inplace=True)

df_punishment = df.copy()
df_punishment['event_type'] = 'punishment'
df_punishment['event_id'] = 1
df_punishment.rename(columns={'punishment': 'event_value_raw', 'punishment_scaled': 'event_value'}, inplace=True)

# Concatenate the two dataframes
df = pd.concat([df_contribution, df_punishment])

# Make sure the data is still sorted correctly after the concatenation
df = df.sort_values(['perm_group', 'episode_id', 'new_player_id', 'round_number', 'event_id'])

In [20]:
input_columns = ['player_id_0', 'player_id_1', 'player_id_2', 'player_id_3',
                 'round_number_sin', 'round_number_cos', 'decay_feature', 'event_value', 'event_id']
query_columns = ['player_id_0', 'player_id_1', 'player_id_2', 'player_id_3','round_number_sin', 'round_number_cos', 'decay_feature', 'event_id']
target_column = 'event_value_raw'

unique_eps = df['episode_id'].unique()

# Split the perm_groups into train and validation groups
train_eps, val_eps = train_test_split(unique_eps, test_size=0.2)

# Create training and validation dataframes based on the split perm_groups
train_df = df[df['episode_id'].isin(train_eps)]
val_df = df[df['episode_id'].isin(val_eps)]

# Create your Dataset
train_dataset = MyDataset(train_df, input_columns, target_column, query_columns)
val_dataset = MyDataset(val_df, input_columns, target_column, query_columns)

# Save the datasets

folder = os.path.join(basedir, dataset_folder)

# Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

train_file = os.path.join(folder, 'train_dataset.pt')
val_file = os.path.join(folder, 'val_dataset.pt')
torch.save(train_dataset, train_file)
torch.save(val_dataset, val_file)

In [21]:
train_eps

array([ 98,  80,  78,  96,  56, 103, 104, 117,  61, 111, 102,  68,  91,
        63, 124,  43,  87,  51,  76, 101,  45,  75,  79,  77, 123,  95,
       112,  62,  44,  93, 116,  60, 120, 110,  49,  46, 119,  57, 121,
        48,  99,  65,  58, 106,  67,  41,  53, 113,  54,  92,  66,  64,
        40, 125, 115,  72,  69,  84, 105, 118,  94,  71,  82, 108,  97,
        83, 107,  55])

In [22]:
val_eps

array([ 74,  50,  70,  88, 100,  42,  52,  90,  89, 109,  47, 114,  59,
        86,  73, 122,  81,  85])

In [23]:
train_dataset.queries.shape[-1]
train_dataset.data.shape[-1]
(train_dataset.targets).max().item() + 1

31

In [24]:
train_dataset.data.shape

torch.Size([1632, 64, 9])

In [25]:
train_dataset.queries.shape

torch.Size([1632, 64, 8])

In [31]:
train_dataset.targets[:,:6]

tensor([[20,  1, 20,  2,  0,  1],
        [20,  0, 20,  0,  4, 10],
        [20,  8,  0,  0,  5,  1],
        ...,
        [ 5,  5,  5, 10, 10,  0],
        [12,  0, 12,  0,  9,  0],
        [10,  0, 10,  0, 10,  0]])

In [32]:
train_dataset.data[:,:6,-2] * 20

tensor([[ 0., 20.,  1., 20.,  2.,  0.],
        [ 0., 20.,  0., 20.,  0.,  4.],
        [ 0., 20.,  8.,  0.,  0.,  5.],
        ...,
        [ 0.,  5.,  5.,  5., 10., 10.],
        [ 0., 12.,  0., 12.,  0.,  9.],
        [ 0., 10.,  0., 10.,  0., 10.]])